# Generate Daily Market Status:

## Tools & Srvices Used:

| ID  | Tool Name | Docs/ Reference |
| --- | --------- | --------------- |
| 1. | [langchain](https://docs.langchain.com/oss/python/langchain/overview) | 1. Build Agents |
| 2. | [groq](https://console.groq.com/docs/quickstart) | |


## Working Environment Setup:

In [4]:
! pip install -qU langchain langchain-groq  # -q: quite output, -U: upgrade
! echo -e "Installed langchain and other dependencies\n"  # -e: enable backslash escapes
# ! pip freeze | grep -iE "langchain|groq"  # -i: ignore case, -E: match either langchain or anthropic

Installed langchain and other dependencies



- Now you can build chains and agents that can perform multi-step tasks. 
- This chain combines a prompt that tells the model what information to extract, 
- a parser that ensures the output follows a specific JSON format, and llama-3.3-70b-versatile to do the actual text processing.

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json


user_prompt = ChatPromptTemplate.from_messages([
    ("system", """Extract stock details into JSON with this structure:
            {{
                "date": "date here in YYYY-MM-DD format",
                "stock_name": "stock name here",
                "last_price": stock_last_price_here_without_currency_symbol
            }}"""),
     ("user", "{input}")
])

llama_llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.1)  # llama-3.1-8b-instant , llama-3.3-70b-versatile

output_parser = JsonOutputParser(pydantic_object={
    "type": "object",
    "properties": {
        "date": {"type": "string", "description": "The date in YYYY-MM-DD format."},
        "stock_name": {"type": "string", "description": "The name of the stocks."},
        "last_price": {"type": "number", "description": "The last price of the stock."}
    }
})

chain = user_prompt | llama_llm | output_parser 

def parse_stock_info(description: str):
    response = chain.invoke({"input": description})
    return response


description = "The stock Reliance Industries Ltd (NSE: RELIANCE) closed at ₹1,554.30 on 12th December, 2025."

parse_stock_info(description)

{'date': '2025-12-12',
 'stock_name': 'Reliance Industries Ltd',
 'last_price': 1554.3}

In [ ]:
def load_json_as_docs(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # Convert each item into a Document (adjust keys as per your JSON structure)
    if isinstance(data, list):
        docs = [Document(page_content=json.dumps(item), metadata={"source": json_file_path}) for item in data]
    else:
        docs = [Document(page_content=json.dumps(data), metadata={"source": json_file_path})]
    return docs

docs = load_json_as_docs('./your_data.json')